In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# --- CONFIGURACIÓN ESTÉTICA PROFESIONAL ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 11

# --- 1. PREPARACIÓN DE DATOS (ETL) ---
def obtener_dataset_maestro():
    """
    Carga y fusiona las tablas aplicando corrección de codificación (encoding)
    para archivos provenientes de Excel/Windows.
    """
    try:
        # Definimos el encoding correcto para español/Excel
        codificacion_excel = 'latin-1' # También puedes probar 'ISO-8859-1' o 'cp1252'
        
        print("🔄 Cargando archivos con codificación especial...")
        
        # Agregamos el parámetro encoding=... a cada lectura
        df_cli = pd.read_csv('clientes.xlsx - clientes.csv.csv', encoding=codificacion_excel)
        df_det = pd.read_csv('detalle_ventas.xlsx - detalle_ventas.csv.csv', encoding=codificacion_excel)
        df_prod = pd.read_csv('productos.xlsx - productos.csv.csv', encoding=codificacion_excel)
        df_ven = pd.read_csv('ventas.xlsx - ventas.csv.csv', encoding=codificacion_excel)
        
        # Conversión de fechas
        df_ven['fecha'] = pd.to_datetime(df_ven['fecha'])
        
        print("✅ Archivos cargados. Iniciando fusión...")
        
        # Fusión Estratégica
        # 1. Detalle + Ventas
        df = pd.merge(df_det, df_ven, on='id_venta', how='inner')
        # 2. + Productos 
        df = pd.merge(df, df_prod, on='id_producto', how='left', suffixes=('', '_prod_dup'))
        # 3. + Clientes 
        df = pd.merge(df, df_cli, on='id_cliente', how='left', suffixes=('', '_cli_dup'))
        
        print(f"✅ Dataset Maestro listo: {df.shape[0]} registros procesados.")
        return df
        
    except Exception as e:
        print(f"❌ Error persistente: {e}")
        return None

# --- 2. NUEVAS TAREAS DE ANÁLISIS ---

def tarea_analisis_categorias(df):
    """
    Tarea: Visualizar el rendimiento por Categoría de Producto.
    Responde: ¿Qué rubro aporta más facturación a la empresa?
    """
    print("\n📦 Ejecutando Análisis por Categoría...")
    
    # Agrupación
    ventas_cat = df.groupby('categoria')['importe'].sum().sort_values(ascending=False)
    
    # Visualización
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=ventas_cat.index, y=ventas_cat.values, palette='viridis', hue=ventas_cat.index, legend=False)
    
    plt.title('Ingresos Totales por Categoría de Producto', fontsize=16, fontweight='bold')
    plt.ylabel('Facturación ($)')
    plt.xlabel('Categoría')
    
    # Añadir etiquetas de datos encima de las barras
    for i, v in enumerate(ventas_cat.values):
        ax.text(i, v + (v*0.01), f"${v:,.0f}", ha='center', fontweight='bold')
        
    plt.show()

def tarea_analisis_medios_pago(df):
    """
    Tarea: Distribución de las Ventas por Medio de Pago.
    Responde: ¿Cuál es el canal de cobro más utilizado?
    """
    print("\n💳 Ejecutando Análisis de Medios de Pago...")
    
    # Contamos cuántas transacciones hubo por medio de pago (usando id_venta único)
    # Primero quitamos duplicados de id_venta para no contar el mismo pago varias veces por cada producto
    pagos_unicos = df.drop_duplicates(subset='id_venta')['medio_pago'].value_counts()
    
    # Visualización: Gráfico de Pastel (Donut Chart)
    plt.figure(figsize=(8, 8))
    colores = sns.color_palette('pastel')[0:5]
    
    plt.pie(pagos_unicos, labels=pagos_unicos.index, autopct='%1.1f%%', 
            startangle=140, colors=colores, pctdistance=0.85, explode=[0.05]*len(pagos_unicos))
    
    # Dibujar círculo blanco en el centro para hacerlo "Donut"
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    
    plt.title('Preferencia de Medios de Pago (Transacciones)', fontsize=16, fontweight='bold')
    plt.show()

def tarea_analisis_geografico(df):
    """
    Tarea: Ventas por Ciudad.
    Responde: ¿Dónde están ubicados nuestros mejores compradores?
    """
    print("\n📍 Ejecutando Análisis Geográfico...")
    
    ventas_ciudad = df.groupby('ciudad')['importe'].sum().sort_values(ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x=ventas_ciudad.values, y=ventas_ciudad.index, palette='coolwarm', hue=ventas_ciudad.index, legend=False)
    
    plt.title('Ranking de Ventas por Ciudad', fontsize=16, fontweight='bold')
    plt.xlabel('Total Vendido ($)')
    plt.ylabel('Ciudad')
    plt.grid(axis='x', linestyle='--')
    plt.show()

# --- 3. EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    df_maestro = obtener_dataset_maestro()
    
    if df_maestro is not None:
        # Tarea A: Categorías
        tarea_analisis_categorias(df_maestro)
        
        # Tarea B: Medios de Pago
        tarea_analisis_medios_pago(df_maestro)
        
        # Tarea C: Geografía
        tarea_analisis_geografico(df_maestro)

🔄 Cargando archivos con codificación especial...
❌ Error persistente: [Errno 2] No such file or directory: 'clientes.xlsx - clientes.csv.csv'
